In [2]:
import os
import pandas as pd

df = pd.read_csv("zoom_logs/test.csv")

In [3]:
df

,이름(원래 이름),사용자 이메일,참가 시간,나간 시간,기간(분),게스트,대기실
0,jun datachef,official.datachef@gmail.com,2024/03/13 09:55:29 PM,2024/03/13 10:56:09 PM,61,아니요,아니요
1,이정욱_53,NaN,2024/03/13 09:56:20 PM,2024/03/13 10:56:09 PM,60,예,아니요
2,정유현_81 (정유현),NaN,2024/03/13 09:56:23 PM,2024/03/13 10:56:09 PM,60,예,아니요
3,이가영22,NaN,2024/03/13 09:56:25 PM,2024/03/13 10:56:09 PM,60,예,아니요
4,김어진_25 (김어진[재학 / 경영학부]),NaN,2024/03/13 09:56:26 PM,2024/03/13 10:56:09 PM,60,예,아니요
...,...,...,...,...,...,...,...
134,사회학과/권수민,NaN,2024/03/13 10:10:55 PM,2024/03/13 10:30:12 PM,20,예,아니요
135,서정인94 (서정인1_2021075387),NaN,2024/03/13 10:14:49 PM,2024/03/13 10:56:09 PM,42,예,아니요
136,장일준_74,NaN,2024/03/13 10:21:33 PM,2024/03/13 10:56:09 PM,35,예,아니요
137,사회학과/권수민,NaN,2024/03/13 10:30:41 PM,2024/03/13 10:39:22 PM,9,예,아니요
